<a href="https://colab.research.google.com/github/etarazonav/650044-ABD-ULIMA/blob/main/Notebooks/ABD_MLlib_Clasificacion_Arboles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a0/Universidad_de_Lima_logo.png/220px-Universidad_de_Lima_logo.png"  width="120" />  MLlib: Clasificación (II)
**Profesor:** Enver G. Tarazona Vargas <br>
**Curso:** Analítica con Big Data <br>
**FACULTAD DE INGENIERÍA - CARRERA DE INGENIERÍA DE SISTEMAS**<br>

# Ejemplo 2: Árboles de Decisión y Derivados


Se utilizará un conjunto de datos para clasificar algunas universidades como privadas o públicas, con base en los siguientes atributos:
* Apps: Número de aplicaciones (postulaciones) recibidas
* Accept: Número de postulaciones aceptadas
* Enroll: Número de alumnos nuevos inscritos
* Top10perc: Estudiantes nuevos del 10% superior del colegio
* Top25perc: Estudiantes nuevos del 25% superior del colegio
* F.Undergrad: Número de estudiantes de pregrado de tiempo completo
* P.Undergrad: Número de estudiantes a tiempo parcial
* Outstate: Costo de inscripción si no se es del estado
* Room.Board: Costos
* Books: Costos estimados de libros
* Personal: Gasto personal estimado
* PhD: Porcentaje de profesores con Ph.D.
* Terminal: Porcentaje de profesores con grado terminal
* S.F.Ratio: Razón estudiante/profesor
* perc.alumni: Porcentaje de ex-alumnos que realizan donaciones
* Expend: Gasto institucional por estudiante
* Grad.Rate: tasa de graduación

In [ ]:
# Solo si se usa colab
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Carga de archivos
!wget -q https://raw.githubusercontent.com/etarazonav/650044-ABD-ULIMA/refs/heads/main/Datos/college.csv

In [ ]:
# Cargar los datos
df = spark.read.csv('college.csv', inferSchema=True, header=True)

# Esquema de los datos
df.printSchema()

In [ ]:
# Algunos valores
df.show(5)

### Pre-procesamiento de Datos

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Ver las columnas disponibles
#df.columns

In [ ]:
# Se tomará todas las columnas, excepto las dos primeras
assembler = VectorAssembler(inputCols=['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F_Undergrad',
                                       'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD',
                                       'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'],
                            outputCol="Atributos")

# Transformar los datos
df2 = assembler.transform(df)
df2.show(5)

Creación de la columna que se va a predecir. Se encuentra con valores categóricos, pero se requiere que sea numérica, por lo que se utilizará una indexación.

In [ ]:
# Convertir la columna objetivo (Private: Yes/No) de categórica a indexada
from pyspark.ml.feature import StringIndexer

# Objeto que realiza la indización
indexer = StringIndexer(inputCol="Private", outputCol="Privado")
# Aplicar a los datos
df3 = indexer.fit(df2).transform(df2)

df3.show(5)

In [ ]:
# Escoger las columnas necesarias para PySpark
df = df3.select("Atributos", 'Privado')

df.show(5)

In [ ]:
# Hacer la división en datos de entrenamiento y datos de prueba
df_train, df_test = df.randomSplit([0.7,0.3],seed= 666)

### Clasificadores

In [ ]:
from pyspark.ml.classification import (DecisionTreeClassifier,
                                       GBTClassifier,
                                       RandomForestClassifier)
from pyspark.ml import Pipeline

In [ ]:
# Crear los tres modelos
dt = DecisionTreeClassifier(labelCol='Privado',featuresCol='Atributos', predictionCol='Predicción')
rf = RandomForestClassifier(labelCol='Privado',featuresCol='Atributos', predictionCol='Predicción' )
gb = GBTClassifier(labelCol='Privado',featuresCol='Atributos', predictionCol='Predicción')

In [ ]:
# Entrenar los modelos
modelo_DT = dt.fit(df_train)
modelo_RF = rf.fit(df_train)
modelo_GB = gb.fit(df_train)

### Inspección de Modelos

In [ ]:
preds_DT = modelo_DT.transform(df_test)

#preds_DT.printSchema()
preds_DT.select("Atributos", "Privado", "Predicción").show(5)

In [ ]:
preds_RF = modelo_RF.transform(df_test)

preds_RF.select("Atributos", "Privado", "Predicción").show(5)

In [ ]:
preds_GB = modelo_GB.transform(df_test)

preds_GB.select("Atributos", "Privado", "Predicción").show(5)

### Métricas de Evaluación

In [ ]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

# Evaluador: usando "exactitud"
evaluadorEX = MulticlassClassificationEvaluator(labelCol="Privado",
                                                predictionCol="Predicción",
                                                metricName='accuracy')

# Evaluador: usando AUC
evaluadorAUC = BinaryClassificationEvaluator(labelCol="Privado",
                                             rawPredictionCol="Predicción",
                                             metricName="areaUnderROC")

In [ ]:
# Mëtricas con árboles de decisión
exactitud_dt = evaluadorEX.evaluate(preds_DT)
auc_dt = evaluadorAUC.evaluate(preds_DT)

print("Usando Árboles de decisión: exactitud={}, AUC={:.3f}".format(exactitud_dt, auc_dt))

In [ ]:
# Mëtricas con random forest
exactitud_rf = evaluadorEX.evaluate(preds_RF)
auc_rf = evaluadorAUC.evaluate(preds_RF)

print("Usando Random Forest: exactitud={:3f}, AUC={:.3f}".format(exactitud_rf, auc_rf))

In [ ]:
# Mëtricas con gradient boosting
exactitud_gb = evaluadorEX.evaluate(preds_GB)
auc_gb = evaluadorAUC.evaluate(preds_GB)

print("Usando Gradient Boosting: exactitud={:3f}, AUC={:.3f}".format(exactitud_gb, auc_gb))

In [ ]:
modelo_RF.featureImportances

In [ ]:
modelo_GB.featureImportances

In [ ]:
modelo_DT.featureImportances